# Playground to setup the code

In [4]:
! pip install /disk1/ariane/vscode/enzyme-tk/dist/enzymetk-0.0.1.tar.gz

Processing /disk1/ariane/vscode/enzyme-tk/dist/enzymetk-0.0.1.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for enzymetk: filename=enzymetk-0.0.1-py3-none-any.whl size=43581 sha256=a9deb8d8d3cd64847ef70e4f012a4b449fa3d87d879e4c70a0ed0dc97cf4c5f7
  Stored in directory: /disk1/ariane/.cache/pip/wheels/a6/2d/27/28efb3ba947af52668dfc32efaa865f4d4960946d7bf6c808d
Successfully built enzymetk
  Attempting uninstall: enzymetk
    Found existing installation: enzymetk 0.0.1
    Uninstalling enzymetk-0.0.1:
      Successfully uninstalled enzymetk-0.0.1


In [5]:
# Read in the sequences and then filter to only have ones with an EC number
import pandas as pd
from Bio import SeqIO
df = pd.read_csv('data/swissprot_info.tsv', sep='\t')
df = df[df['EC number'].notna()]
# Now let's filter the sequneces that have these ec numbers

fasta = SeqIO.parse('data/uniprot_sprot.fasta', 'fasta')
enzymes = set(df['Entry'].values)
with open('data/uniprot_sprot_enzymes.fasta', 'w') as f: 
    for record in fasta:
        enzyme_id = record.id.split('|')[1]
        if enzyme_id in enzymes:
            f.write(f'>{enzyme_id}\n{record.seq}\n')




In [2]:
import pandas as pd
from enzymetk.sequence_search_blast import BLAST
from enzymetk.similarity_foldseek_step import FoldSeek
from enzymetk.annotateEC_proteinfer_step import ProteInfer
from enzymetk.annotateEC_CLEAN_step import CLEAN
from enzymetk.save_step import Save
import pandas as pd

id_col = 'id'
label_col = 'label'
seq_col = 'sequence'    


og_df = pd.read_parquet('/disk1/share/data/OG/data/train-00000-of-00110.parquet')
og_df


,CDS_position_ids,IGS_position_ids,CDS_ids,IGS_ids,CDS_seqs,IGS_seqs,CDS_orientations
0,"[1, 3, 5, 7, 9, 10, 11, 12, 14, 16, 17, 19, 21...","[0, 2, 4, 6, 8, 13, 15, 18, 20, 22, 24, 26, 29...",[2961237444|Ga0149631_01|CDS|2961237445|-|1061...,[2961237444|Ga0149631_01|IG|IG_000001|+|1043:1...,[MNQLDALKQLTTVVADSGDIESIRQFEPQDATTNPSLILKAAALP...,"[AACGAAACTCCAGTACGG, TGATAAGCTCCTTGACCAAGCTCAA...","[False, False, True, False, True, True, True, ..."
1,"[0, 2, 3, 4, 5, 7, 9, 11, 13, 15, 17, 19, 21, ...","[1, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, ...",[2961237444|Ga0149631_01|CDS|2961237530|+|8547...,[2961237444|Ga0149631_01|IG|IG_000064|+|87422:...,[ATWYLTRAYVDQAAGRSWAALENVTRARALAPSDAGIQAYYVTLL...,"[AGGAATAATC, TACAGTCCATTGTCGTGATCATTGCGGCGATAG...","[True, True, True, True, False, True, False, T..."
2,"[0, 2, 4, 5, 6, 7, 9, 10, 12, 14, 16, 17, 18, ...","[1, 3, 8, 11, 13, 15, 19, 21, 23, 25, 28, 30, ...",[2961237444|Ga0149631_01|CDS|2961237823|-|3973...,[2961237444|Ga0149631_01|IG|IG_000289|+|400444...,[MPAAPLASLTEILARRDWQNPACTHYRRLDAHPPFASWRTVEDAR...,[CGGCGATCGTCCAGTCGGCTAGGTGATGCTCACCAAGATGCTGAC...,"[False, False, True, True, True, True, True, T..."
3,"[1, 3, 5, 7, 9, 11, 13, 14, 15, 17, 19, 21, 23...","[0, 2, 4, 6, 8, 10, 12, 16, 18, 20, 22, 24, 26...",[2961237444|Ga0149631_01|CDS|2961238380|+|9477...,[2961237444|Ga0149631_01|IG|IG_000747|+|947343...,[MDFQAIKNTARQARRYVVTMNHKAGKGHTGADLSEIDIICTLYMA...,[TCGAAAGACTCCTGCACCTGTCAGTTCCCGCCGGGCGGTTTGCCA...,"[True, True, True, True, True, True, True, Tru..."
4,"[0, 1, 3, 5, 7, 9, 10, 12, 14, 16, 18, 20, 22,...","[2, 4, 6, 8, 11, 13, 15, 17, 19, 21, 23, 25, 2...",[2961237444|Ga0149631_01|CDS|2961238402|+|9785...,[2961237444|Ga0149631_01|IG|IG_000768|+|979638...,[SESEKQQVSALSQLAAGLAGGLATGDTAGAVTGGQAGKNAVENNA...,"[GGTAGCGGGGCTCACGGCATCCCA, T, AGTAACAAAATCCCGG...","[True, True, True, True, True, True, True, Tru..."
...,...,...,...,...,...,...,...
56417,"[1, 2, 4, 6, 8, 10, 12]","[0, 3, 5, 7, 9, 11, 13]",[2828558644|Ga0392896_0312|CDS|2828559967|+|51...,[2828558644|Ga0392896_0312|IG|IG_000001|+|489:...,[MVNAQGESANTQVREMAANVNGASVENARRIEHTQLILPVDTRTP...,"[GTGGAAGCCAAACGGGCCAGG, ATATTAAGGTGTTGGAGTAGTA...","[True, True, False, False, True, False, False]"
56418,"[1, 2, 4, 6, 8]","[0, 3, 5, 7, 9]",[2828558644|Ga0392896_0320|CDS|2828559998|+|63...,[2828558644|Ga0392896_0320|IG|IG_000001|+|476:...,[MRSRNDDHQTVFSAICSHFEIERSGNVLGSVLLLFWFVGCNGSAA...,[GATCTCAAGACAATCTTGTGCGTATTTCTCATGTGTTGCGGCGCC...,"[True, False, False, False, True]"
56419,"[1, 3, 5, 7]","[0, 2, 4, 6, 8]",[2828558644|Ga0392896_0324|CDS|2828560013|+|15...,[2828558644|Ga0392896_0324|IG|IG_000001|+|1519...,[MVHLGCLDKVLKFHRKFCVPARLSTPWIFARRPKSKYRHKLANEE...,"[GGAGTGTCCTCGACCG, CTTCGGCAACTAGGAGGAAATAATAGA...","[True, True, False, False]"
56420,"[1, 3, 5, 7, 9, 10, 12, 14, 16, 18]","[0, 2, 4, 6, 8, 11, 13, 15, 17, 19]",[2828558644|Ga0392896_0325|CDS|2828560019|-|60...,[2828558644|Ga0392896_0325|IG|IG_000001|+|427:...,[MKPIRFHPVFQHYLWGGHRLANELGKPVSSADICAESWEVVDHRD...,[ATCCAAACTCCGCGGGCTGGATCAACGAGAGGGTTTGACCAAAAG...,"[False, True, True, True, True, True, False, F..."


In [8]:
from sciutil import SciUtil
import os

u = SciUtil()

def pipeline(run_name, input_df, id_col, seq_col, output_folder):
    input_df << (BLAST(id_col, seq_col, database='data/uniprot_sprot.fasta', args=['--ultra-sensitive']) >> Save(os.path.join(output_folder, f'{run_name}_blast.pkl')))

    # After this we summarize the BLAST file
    df = pd.read_pickle(os.path.join(output_folder, f'{run_name}_blast.pkl'))
    df.sort_values(by='sequence identity', ascending=False, inplace=True)
    df = df.drop_duplicates(subset=['query'])
    # Now we would check which ones were unable to be found
    # existing seqs
    df['annotation'] = 'BLAST'
    input_df = input_df[~input_df[id_col].isin(df['query'])]
    u.dp([len(df), 'identified by BLAST. Continuing with ', len(input_df)])

    # Now we need to do foldseek against these remaining sequences
    reference_database = '/disk1/share/software/foldseek/structures/pdb/pdb'
    input_df << (FoldSeek(id_col, seq_col, reference_database, query_type='seqs') >> Save(os.path.join(output_folder, f'{run_name}_foldseek.pkl')))

    min_fident = 0.1
    foldseek_df = pd.read_pickle(os.path.join(output_folder, f'{run_name}_foldseek.pkl'))
    foldseek_df.sort_values(by='fident', ascending=False, inplace=True)
    foldseek_df = foldseek_df.drop_duplicates(subset=['query'])
    foldseek_df = foldseek_df[foldseek_df['fident'] > min_fident]
    # Now we would check which ones were unable to be found
    # existing seqs
    foldseek_df['annotation'] = 'foldseek'
    input_df = input_df[~input_df[id_col].isin(foldseek_df['query'])]
    # Now we need to do foldseek against these remaining sequences
    u.dp([len(foldseek_df), 'identified by FoldSEEK. Continuing with ', len(input_df)])

    # Running proteInfer 
    input_df << (ProteInfer(id_col, seq_col, proteinfer_dir='/disk1/ariane/vscode/enzyme-tk/enzymetk/conda_envs/proteinfer/') >> Save(os.path.join(output_folder, f'{run_name}_proteinfer.pkl')))

    proteinfer_df = pd.read_pickle(os.path.join(output_folder, f'{run_name}_proteinfer.pkl'))
    min_fident = 0.5
    proteinfer_df.sort_values(by='confidence', ascending=False, inplace=True)
    proteinfer_df = proteinfer_df.drop_duplicates(subset=['sequence_name'])
    proteinfer_df = proteinfer_df[proteinfer_df['confidence'] > min_fident]
    # Now we would check which ones were unable to be found
    # existing seqs
    proteinfer_df['annotation'] = 'proteinfer'
    # Continue with all sequences since we are unsure with ML
    #input_df = input_df[~input_df['id'].isin(proteinfer_df['sequence_name'])]
    u.dp([len(proteinfer_df), 'identified by ProteInfer. Continuing with ', len(input_df)])

    # CLEAN directory
    clean_dir = '/disk1/share/software/CLEAN/app/'
    input_df << (CLEAN(id_col, seq_col, clean_dir, num_threads=1) >> Save(os.path.join(output_folder, f'{run_name}_clean.pkl')))




In [10]:
from tqdm import tqdm
input_df = og_df.head(10)
# Then we want to convert the IDs and the sequneces
j = 0

id_col = 'id'
label_col = 'label'
seq_col = 'seq'  
output_folder = '/disk1/ariane/vscode/annotate-e/examples/'
rows = []
 # I think chunking is better since we don't need to do this 
for ids, seqs in tqdm(input_df[['CDS_ids', 'CDS_seqs']].values):
    for i, s_id in enumerate(ids):
        rows.append([f'o{j}s{i}', s_id, seqs[i]])

run_name = f'o{j}'
try:
    os.mkdir(os.path.join(output_folder, run_name))
except:
    u.warn_p(['Directory already existed: ', os.path.join(output_folder, run_name)])

input_df = pd.DataFrame(rows, columns=['id', 'cds_id', 'seq'])
# Save the input df to a file as well since then we can extract this as well 
pipeline(run_name, input_df, id_col, seq_col, os.path.join(output_folder, run_name))


  0%|          | 0/10 [00:00<?, ?it/s]diamond v2.1.11.165 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: data/uniprot_sprot.fasta
Opening the database file...  [0.002s]
Loading sequences... 

--------------------------------------------------------------------------------
    Directory already existed: 	/disk1/ariane/vscode/annotate-e/examples/o0	    
--------------------------------------------------------------------------------


 [1.179s]
Masking sequences...  [0.196s]
Writing sequences...  [0.239s]
Hashing sequences...  [0.072s]
Loading sequences...  [0s]
Writing trailer...  [0.008s]
Closing the input file...  [0s]
Closing the database file...  [0.002s]

Database sequences  572970
  Database letters  207621817
     Database hash  b878f03de9798961f4c9ce7e8ee8ed9f
        Total time  1.700000s


['diamond', 'blastp', '--ultra-sensitive', '-d', '/tmp/tmpm8ykfdh4/CfVLt1O6Mt_db', '-q', '/tmp/tmpm8ykfdh4/CfVLt1O6Mt_query.fasta', '-o', '/tmp/tmpm8ykfdh4/CfVLt1O6Mt_matches.tsv']
--------------------------------------------------------------------------------
Running command	diamond blastp --ultra-sensitive -d /tmp/tmpm8ykfdh4/CfVLt1O6Mt_db -q /tmp/tmpm8ykfdh4/CfVLt1O6Mt_query.fasta -o /tmp/tmpm8ykfdh4/CfVLt1O6Mt_matches.tsv	
--------------------------------------------------------------------------------


diamond v2.1.11.165 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: /tmp/tmpm8ykfdh4
#Target sequences to report alignments for: 25
Opening the database...  [0.068s]
Database: /tmp/tmpm8ykfdh4/CfVLt1O6Mt_db (type: Diamond database, sequences: 572970, letters: 207621817)
Block size = 400000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0s]
Masking queries...  [0.018s]
Algorithm: Double-indexed
Building query histograms...  [0.106s]
Seeking in database...  [0s]
Loading reference sequences...  [0.367s]
Masking reference...  [0.183s]
Initializing temporary storage...  [0.001s]
Building reference histograms...  [0.772s]
Alloc

...

I0328 14:23:40.331110 140602939057984 proteinfer.py:392] Running with 1 ensemble elements
I0328 14:23:40.331271 140602939057984 proteinfer.py:189] Parsing input from /tmp/tmp_mbsce4kproteinfer.fasta

Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.04it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  3.79it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  4.08it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  3.93it/s]


Progress: 100%|██████████| 3/3 [00:00<00:00,  3.22it/s]
I0328 14:23:44.192078 140602939057984 proteinfer.py:340] Writing output to /tmp/tmp_mbsce4kproteinfer.txt




--------------------------------------------------------------------------------
                                    Output:	                                    
--------------------------------------------------------------------------------




--------------------------------------------------------------------------------
Error:	I0328 14:23:40.331110 140602939057984 proteinfer.py:392] Running with 1 ensemble elements
I0328 14:23:40.331271 140602939057984 proteinfer.py:189] Parsing input from /tmp/tmp_mbsce4kproteinfer.fasta

Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.04it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  3.79it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  4.08it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  3.93it/s]


Progress: 100%|██████████| 3/3 [00:00<00:00,  3.22it/s]
I0328 14:23:44.192078 140602939057984 proteinfer.py:340] Writing output to /tmp/tmp_mbsce4kproteinfer.


100%|██████████| 5242/5242 [00:00<00:00, 34588.58it/s]

0it [00:00, ?it/s]
2it [00:00, 63.64it/s]


 10%|█         | 1/10 [01:05<09:53, 65.95s/it]diamond v2.1.11.165 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: /tmp/tmp5603dh7s
#Target sequences to report alignments for: 25
Opening the database... 
Error opening file data/uniprot_sprot.fasta: No such file or directory

 10%|█         | 1/10 [01:05<09:53, 65.96s/it]

--------------------------------------------------------------------------------
                                    Output:	                                    
--------------------------------------------------------------------------------
Transferred model to GPU
Read data/inputs/46TxqxmRr2.fasta with 2 sequences
Processing 1 of 1 batches (2 sequences)
The embedding sizes for train and test: torch.Size([241025, 128]) torch.Size([2, 128])
Calculating eval distance map, between 2 test ids and 5242 train EC cluster centers


--------------------------------------------------------------------------------
Error:	
100%|██████████| 5242/5242 [00:00<00:00, 34588.58it/s]

0it [00:00, ?it/s]
2it [00:00, 63.64it/s]

	
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
              Time for command to run (min): 	0.2778783621188874	               
-----------------------------------

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmp5603dh7s/jfj2zulq9s_matches.tsv'

In [4]:
pd.read_pickle('/disk1/ariane/vscode/annotate-e/examples/clean_test.pkl')


,0
0,"o1s201,EC:2.6.1.1/0.0016"
1,"o2s34,EC:2.7.7.48/0.0950"
2,"o2s362,EC:2.6.1.1/0.0094"
3,"o2s446,EC:3.4.22.65/0.2084"
4,"o2s453,EC:3.4.21.84/0.0015,EC:4.99.1.10/0.0013..."
5,"o4s74,EC:3.4.24.50/0.0374"
6,"o4s209,EC:2.6.1.1/0.0012,EC:3.4.24.50/0.0011"
7,"o4s327,EC:2.6.1.1/0.0010"
8,"o6s62,EC:4.1.2.13/0.0002"
9,"o7s187,EC:3.4.22.65/0.0013"


In [5]:
pd.read_pickle('/disk1/ariane/vscode/annotate-e/examples/pdb_files.pkl')


,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits
0,o2s307,3r23-assembly1.cif.gz_A,0.158,250,194,0,30,279,61,291,0.000005,324
1,o2s307,3r23-assembly1.cif.gz_B,0.147,261,208,0,20,280,44,288,0.000007,318
2,o2s307,7dro-assembly1.cif.gz_A,0.120,229,162,0,51,279,114,298,0.000014,306
3,o2s307,7dro-assembly1.cif.gz_B,0.117,240,176,0,39,278,97,296,0.000014,306
4,o2s307,2pvp-assembly1.cif.gz_A,0.118,226,177,0,57,282,128,329,0.000018,302
...,...,...,...,...,...,...,...,...,...,...,...,...
98817,o6s191,6fn9-assembly1.cif.gz_A,0.077,285,187,0,108,392,21,224,9.883000,75
98818,o6s191,3lw1-assembly1.cif.gz_A-2,0.132,132,102,0,108,226,27,158,9.883000,75
98819,o6s191,6grj-assembly1.cif.gz_F,0.116,404,274,0,15,418,3,313,9.883000,75
98820,o6s191,3qtn-assembly1.cif.gz_B,0.101,431,283,0,119,549,6,321,9.883000,75


In [7]:
pd.read_pickle('/disk1/ariane/vscode/annotate-e/examples/blast_test.pkl')


,query,target,sequence identity,length,mismatch,gapopen,query start,query end,target start,target end,e-value,bitscore
0,o0s0,sp|Q6D8W0|TAL1_PECAS,82.9,316,54,0,1,316,1,316,4.040000e-186,518.0
1,o0s0,sp|Q2NS86|TAL_SODGM,82.0,316,57,0,1,316,1,316,3.170000e-183,510.0
2,o0s0,sp|Q57LN7|TAL2_SALCH,72.8,316,86,0,1,316,1,316,3.740000e-168,472.0
3,o0s0,sp|Q8Z4T0|TALA_SALTI,72.8,316,86,0,1,316,1,316,3.740000e-168,472.0
4,o0s0,sp|Q8ZN83|TALA_SALTY,72.8,316,86,0,1,316,1,316,3.740000e-168,472.0
...,...,...,...,...,...,...,...,...,...,...,...,...
46684,o9s77,sp|P09163|YJAB_ECOLI,46.1,128,68,1,1,128,2,128,1.230000e-31,112.0
46685,o9s77,sp|Q7VG78|GUAA_HELHP,39.8,118,66,3,15,128,158,274,1.870000e-15,74.7
46686,o9s77,sp|P37664|YIAC_ECOLI,30.3,122,83,2,4,124,2,122,2.790000e-12,62.8
46687,o9s77,sp|A0A0S0MX59|AT_BPPAM,43.1,58,29,2,75,128,71,128,2.520000e-06,47.0


In [6]:
pd.read_pickle('/disk1/ariane/vscode/annotate-e/examples/proteinfer.pkl')


,sequence_name,predicted_label,confidence,description
0,o9s48,Pfam:PF06568,0.79,Domain of unknown function (DUF1127)
1,o1s144,Pfam:PF03911,1.00,Sec61beta family
2,o2s81,Pfam:PF01783,1.00,Ribosomal L32p protein family
3,o2s346,Pfam:PF08041,1.00,PetM family of cytochrome b6f complex subunit 7
4,o4s20,Pfam:PF04866,0.83,Rotavirus non-structural protein 6
...,...,...,...,...
402,o7s361,GO:0008150,0.70,biological_process
403,o8s247,GO:0042597,0.99,periplasmic space
404,o9s34,GO:0016020,1.00,membrane
405,o9s34,GO:0005575,0.99,cellular_component


In [ ]:
input_df << (BLAST(id_col, seq_col, database='data/uniprot_sprot.fasta', args=['--ultra-sensitive']) >> Save('tmp/blast_test.pkl'))
# After this we summarize the file
df = pd.read_pickle('tmp/blast_test.pkl')

diamond v2.1.11.165 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: data/uniprot_sprot.fasta
Opening the database file...  [0.002s]
Loading sequences...  [1.161s]
Masking sequences...  [0.18s]
Writing sequences...  [0.246s]
Hashing sequences...  [0.072s]
Loading sequences...  [0s]
Writing trailer...  [0.008s]
Closing the input file...  [0.003s]
Closing the database file...  [0.002s]

Database sequences  572970
  Database letters  207621817
     Database hash  b878f03de9798961f4c9ce7e8ee8ed9f
        Total time  1.677000s


['diamond', 'blastp', '-d', '/tmp/tmpcekx_nb0/ZgQHsRyfyt_db', '-q', '/tmp/tmpcekx_nb0/ZgQHsRyfyt_query.fasta', '-o', '/tmp/tmpcekx_nb0/ZgQHsRyfyt_matches.tsv']
--------------------------------------------------------------------------------
Running command	diamond blastp -d /tmp/tmpcekx_nb0/ZgQHsRyfyt_db -q /tmp/tmpcekx_nb0/ZgQHsRyfyt_query.fasta -o /tmp/tmpcekx_nb0/ZgQHsRyfyt_matches.tsv	
--------------------------------------------------------------------------------


diamond v2.1.11.165 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: /tmp/tmpcekx_nb0
#Target sequences to report alignments for: 25
Opening the database...  [0.069s]
Database: /tmp/tmpcekx_nb0/ZgQHsRyfyt_db (type: Diamond database, sequences: 572970, letters: 207621817)
Block size = 2000000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0s]
Masking queries...  [0.017s]
Algorithm: Double-indexed
Building query histograms...  [0.05s]
Seeking in database...  [0s]
Loading reference sequences...  [0.356s]
Masking reference...  [0.174s]
Initializing temporary storage...  [0s]
Building reference histograms...  [0.112s]
Allocatin

--------------------------------------------------------------------------------
                                    Output:	                                    
--------------------------------------------------------------------------------

--------------------------------------------------------------------------------
Error:	diamond v2.1.11.165 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: /tmp/tmpcekx_nb0
#Target sequences to report alignments for: 25
Opening the database...  [0.069s]
Database: /tmp/tmpcekx_nb0/ZgQHsRyfyt_db (type: Diamond database, sequences: 572970, letters: 207621817)
Block size = 2000000000
Opening the input file...  [0s]
Opening the output f

In [14]:

df.sort_values(by='sequence identity', ascending=False, inplace=True)
df = df.drop_duplicates(subset=['query'])
# Now we would check which ones were unable to be found
# existing seqs
df['annotation'] = 'BLAST'
input_df = input_df[~input_df['id'].isin(df['query'])]

# Now we need to do foldseek against these remaining sequences


/tmp/ipykernel_4150777/1101129157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='sequence identity', ascending=False, inplace=True)


,id,label,sequence
4,A0A0B8RHP0_LISMN,reference,MKKLLFLGDSVTDAGRDFENDRELGHGYVKIIADQLEQEDVTVINR...


In [4]:
from enzymetk.similarity_foldseek_step import FoldSeek

reference_database = '/disk1/share/software/foldseek/structures/pdb/pdb'
input_df << (FoldSeek('id', 'sequence', reference_database, query_type='seqs',  tmp_dir='tmp/') >> Save('pdb_files.pkl'))


--------------------------------------------------------------------------------
            Running command	foldseek databases ProstT5 weights tmp	             
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                                    Output:	                                    
--------------------------------------------------------------------------------
databases ProstT5 weights tmp 

MMseqs Version:              	10.941cd33
Tsv                          	false
Force restart with latest tmp	false
Remove temporary files       	false
Compressed                   	0
Threads                      	384
Verbosity                    	3


03/27 16:48:02 [NOTICE] Downloading 1 item(s)
[#efda92 1.0MiB/2.0GiB(0%) CN:5 DL:4.2MiB ETA:8m18s]
[#efda92 40MiB/2.0GiB(1%) CN:5 DL:33MiB ETA:1m2s]
[#efda92 105MiB/2.0GiB(4%) CN:5 DL:47MiB ETA:42s]
[#efda92 188MiB/2.0GiB(8%) CN:5 DL

,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits
0,A0A0B8RHP0_LISMN,3rjt-assembly1.cif.gz_B,0.321,203,130,0,2,194,7,209,9.309000e-18,724
1,A0A0B8RHP0_LISMN,5jd3-assembly1.cif.gz_A,0.294,204,136,0,2,194,8,211,3.636000e-17,702
2,A0A0B8RHP0_LISMN,4q9a-assembly1.cif.gz_B,0.330,205,129,0,2,194,9,213,6.754000e-17,692
3,A0A0B8RHP0_LISMN,5bn1-assembly1.cif.gz_A,0.321,203,130,0,2,194,7,209,4.900000e-16,660
4,A0A0B8RHP0_LISMN,4oap-assembly1.cif.gz_A,0.326,203,129,0,2,194,7,209,8.043000e-16,652
...,...,...,...,...,...,...,...,...,...,...,...,...
4280,AXE2_TALPU,3gwz-assembly1.cif.gz_A,0.135,71,54,0,88,158,183,245,9.451000e+00,53
4281,AXE2_TALPU,3oti-assembly1.cif.gz_A,0.102,231,158,0,3,233,196,372,9.451000e+00,53
4282,AXE2_TALPU,3xim-assembly1.cif.gz_C,0.073,180,146,0,74,231,137,316,9.451000e+00,53
4283,AXE2_TALPU,7ung-assembly1.cif.gz_IB,0.060,71,56,0,90,160,138,198,9.451000e+00,53


In [5]:
min_fident = 0.1
foldseek_df = pd.read_pickle('pdb_files.pkl')
foldseek_df.sort_values(by='fident', ascending=False, inplace=True)
foldseek_df = foldseek_df.drop_duplicates(subset=['query'])
foldseek_df = foldseek_df[foldseek_df['fident'] > min_fident]
# Now we would check which ones were unable to be found
# existing seqs
foldseek_df['annotation'] = 'foldseek'
input_df = input_df[~input_df['id'].isin(foldseek_df['query'])]
input_df
# Now we need to do foldseek against these remaining sequences


,id,label,sequence


In [6]:
# Now run them with proteinfer and then also CLEAN
from enzymetk.similarity_foldseek_step import FoldSeek

reference_database = '/disk1/share/software/foldseek/structures/pdb/pdb'
input_df << (FoldSeek('id', 'sequence', reference_database, query_type='seqs',  tmp_dir='tmp/') >> Save('pdb_files.pkl'))


,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,annotation
2647,AXE2_TALPU,1g66-assembly1.cif.gz_A,1.000,206,0,0,28,233,1,206,4.589000e-36,1366,foldseek
803,AXE2_GEOSE,4jhl-assembly1.cif.gz_A,1.000,219,0,0,1,219,1,219,8.291000e-37,1453,foldseek
2,A0A0B8RHP0_LISMN,4q9a-assembly1.cif.gz_B,0.330,205,129,0,2,194,9,213,6.754000e-17,692,foldseek
1665,AXE7A_XYLR2,1l7a-assembly1.cif.gz_A,0.272,307,223,0,133,439,11,317,1.121000e-26,1134,foldseek


In [3]:
from enzymetk.annotateEC_proteinfer_step import ProteInfer


input_df << (ProteInfer('id', 'sequence', proteinfer_dir='/disk1/ariane/vscode/enzyme-tk/enzymetk/conda_envs/proteinfer/') >> Save('proteinfer.pkl'))

proteinfer_df = pd.read_pickle('proteinfer.pkl')
min_fident = 0.5
proteinfer_df.sort_values(by='confidence', ascending=False, inplace=True)
proteinfer_df = proteinfer_df.drop_duplicates(subset=['sequence_name'])
proteinfer_df = proteinfer_df[proteinfer_df['confidence'] > min_fident]
# Now we would check which ones were unable to be found
# existing seqs
proteinfer_df['annotation'] = 'proteinfer'
input_df = input_df[~input_df['id'].isin(proteinfer_df['sequence_name'])]
input_df



--------------------------------------------------------------------------------
Running command	conda run -n proteinfer python3 /disk1/ariane/vscode/enzyme-tk/enzymetk/conda_envs/proteinfer/proteinfer.py -i /tmp/tmphf0549njproteinfer.fasta -o /tmp/tmphf0549njproteinfer.txt	
--------------------------------------------------------------------------------


I0328 09:46:41.015074 140337427027776 proteinfer.py:392] Running with 1 ensemble elements
I0328 09:46:41.015233 140337427027776 proteinfer.py:189] Parsing input from /tmp/tmphf0549njproteinfer.fasta

Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.00it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]


Progress: 100%|██████████| 3/3 [00:01<00:00,  1.73it/s]
I0328 09:46:45.801395 140337427027776 proteinfer.py:340] Writing output to /tmp/tmphf0549njproteinfer.txt




--------------------------------------------------------------------------------
                                    Output:	                                    
--------------------------------------------------------------------------------




--------------------------------------------------------------------------------
Error:	I0328 09:46:41.015074 140337427027776 proteinfer.py:392] Running with 1 ensemble elements
I0328 09:46:41.015233 140337427027776 proteinfer.py:189] Parsing input from /tmp/tmphf0549njproteinfer.fasta

Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.00it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]


Annotating batches of sequences: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]


Progress: 100%|██████████| 3/3 [00:01<00:00,  1.73it/s]
I0328 09:46:45.801395 140337427027776 proteinfer.py:340] Writing output to /tmp/tmphf0549njproteinfer.

,sequence_name,predicted_label,confidence,description
0,AXE2_TALPU,Pfam:CL0028,1.00,AB_hydrolase
1,AXE2_TALPU,Pfam:CL0028,1.00,AB_hydrolase
2,AXE2_TALPU,Pfam:PF01083,1.00,Cutinase
3,AXE2_TALPU,Pfam:PF01083,1.00,Cutinase
4,AXE2_GEOSE,Pfam:PF13472,1.00,GDSL-like Lipase/Acylhydrolase family
...,...,...,...,...
179,A0A0B8RHP0_LISMN,GO:0009987,0.59,cellular process
180,A0A0B8RHP0_LISMN,GO:0046872,0.57,metal ion binding
181,A0A0B8RHP0_LISMN,GO:0008152,0.55,metabolic process
182,A0A0B8RHP0_LISMN,GO:0042578,0.54,phosphoric ester hydrolase activity


In [4]:

proteinfer_df = pd.read_pickle('proteinfer.pkl')
min_fident = 0.5
proteinfer_df.sort_values(by='confidence', ascending=False, inplace=True)
proteinfer_df = proteinfer_df.drop_duplicates(subset=['sequence_name'])
proteinfer_df = proteinfer_df[proteinfer_df['confidence'] > min_fident]
# Now we would check which ones were unable to be found
# existing seqs
proteinfer_df['annotation'] = 'proteinfer'
input_df = input_df[~input_df['id'].isin(proteinfer_df['sequence_name'])]
input_df

,id,label,sequence


In [3]:
import pandas as pd
from enzymetk.sequence_search_blast import BLAST
from enzymetk.save_step import Save
import pandas as pd

id_col = 'id'
label_col = 'label'
seq_col = 'sequence'    

rows = [['AXE21', 'query', 'MHSKFFAASLLGLGAAAIPLEGVMEKRSCPAIHVFGARETTASPGYGSSSTVVNGVLSAYPGSTAEAINYPACGGQSSCGGASYSSSVAQGIAAVASAVNSFNSQCPSTKIVLVGYSQGGEIMDVALCGGGDPNQGYTNTAVQLSSSAVNMVKAAIFMGDPMFRAGLSYEVGTCAAGGFDQRPAGFSCPSAAKIKSYCDASDPYCCNGSNAATHQGYGSEYGSQALAFVKSKLG'],
        ['AXE22', 'reference', 'MHSKFFAASLLGLGAAAIPLEGVMEKRSCPAIHVFGARETTASPGYGSSSTVVNGVLSAYPGSTAEAINYPACGGQSSCGGASYSSSVAQGIAAVASAVNSFNSQCPSTKIVLVGYSQGGEIMDVALCGGGDPNQGYTNTAVQLSSSAVNMVKAAIFMGDPMFRAGLSYEVGTCAAGGFDQRPAGFSCPSAAKIKSYCDASDPYCCNGSNAATHQGYGSEYGSQALAFVKSKLG'],
        ['AXE23', 'reference', 'MKIGSGEKLLFIGDSITDCGRARPEGEGSFGALGTGYVAYVVGLLQAVYPELGIRVVNKGISGNTVRDLKARWEEDVIAQKPDWVSIMIGINDVWRQYDLPFMKEKHVYLDEYEATLRSLVLETKPLVKGIILMTPFYIEGNEQDPMRRTMDQYGRVVKQIAEETNSLFVDTQAAFNEVLKTLYPAALAWDRVHPSVAGHMILARAFLREIGFEWVRSR'], 
        ['AXE7A4', 'referece', 'MFNFAPKQTTEMKKLLFTLVFVLGSMATALAENYPYRADYLWLTVPNHADWLYKTGERAKVEVSFCLYGMPQNVEVAYEIGPDMMPATSSGKVTLKNGRAVIDMGTMKKPGFLDMRLSVDGKYQHHVKVGFSPELLKPYTKNPQDFDAFWKANLDEARKTPVSVSCNKVDKYTTDAFDCYLLKIKTDRRHSIYGYLTKPKKAGKYPVVLCPPGAGIKTIKEPMRSTFYAKNGFIRLEMEIHGLNPEMTDEQFKEITTAFDYENGYLTNGLDDRDNYYMKHVYVACVRAIDYLTSLPDWDGKNVFVQGGSQGGALSLVTAGLDPRVTACVANHPALSDMAGYLDNRAGGYPHFNRLKNMFTPEKVNTMAYYDVVNFARRITCPVYITWGYNDNVCPPTTSYIVWNLITAPKESLITPINEHWTTSETNYTQMLWLKKQVK'], 
        ['A0A0B8RHP05', 'reference', 'MKKLLFLGDSVTDAGRDFENDRELGHGYVKIIADQLEQEDVTVINRGVSANRVADLHRRIEADAISLQPDVVTIMIGINDTWFSFSRWEDTSVTAFKEVYRVILNRIKTETNAELILMEPFVLPYPEDRKEWRGDLDPKIGAVRELAAEFGATLIPLDGLMNALAIKHGPTFLAEDGVHPTKAGHEAIASTWLEFTK']]
df = pd.DataFrame(rows, columns=[id_col, label_col, seq_col])


In [4]:
from enzymetk.annotateEC_CLEAN_step import CLEAN
from enzymetk.save_step import Save
clean_dir = '/disk1/share/software/CLEAN/app/'
df << (CLEAN(id_col, seq_col, clean_dir, num_threads=1) >> Save(f'/disk1/ariane/vscode/enzyme-tk/examples/clean_test.pkl'))

import pandas as pd
input_df = pd.read_parquet('/disk1/share/data/OG/data/train-00000-of-00110.parquet')
input_df

--------------------------------------------------------------------------------
Running command	cp /tmp/tmpowanhm5zCLEAN_iqmU3lkN7u.fasta /disk1/share/software/CLEAN/app/data/inputs/seCLZigYT8.fasta	
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                                    Output:	                                    
--------------------------------------------------------------------------------

--------------------------------------------------------------------------------
                                    Error:		                                    
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
             Time for command to run (min): 	6.546149961650372e-05	             
---------------------------------------------------------------------


100%|██████████| 5242/5242 [00:00<00:00, 34132.59it/s]

0it [00:00, ?it/s]
5it [00:00, 134.00it/s]




--------------------------------------------------------------------------------
                                    Output:	                                    
--------------------------------------------------------------------------------
Transferred model to GPU
Read data/inputs/seCLZigYT8.fasta with 5 sequences
Processing 1 of 1 batches (5 sequences)
The embedding sizes for train and test: torch.Size([241025, 128]) torch.Size([5, 128])
Calculating eval distance map, between 5 test ids and 5242 train EC cluster centers


--------------------------------------------------------------------------------
Error:	
100%|██████████| 5242/5242 [00:00<00:00, 34132.59it/s]

0it [00:00, ?it/s]
5it [00:00, 134.00it/s]

	
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
              Time for command to run (min): 	0.3043132823503887	               
----------------------------------

,0
0,"AXE21,EC:3.1.1.74/0.7807,EC:3.1.1.72/0.6912"
1,"AXE22,EC:3.1.1.74/0.7807,EC:3.1.1.72/0.6912"
2,"AXE23,EC:3.1.1.72/0.9034"
3,"AXE7A4,EC:3.1.1.72/0.0006"
4,"A0A0B8RHP05,EC:3.1.1.72/0.0055,EC:3.1.1.112/0...."


In [8]:
import pandas as pd
input_df = pd.read_parquet('/disk1/share/data/OG/data/train-00000-of-00110.parquet')
input_df

,CDS_position_ids,IGS_position_ids,CDS_ids,IGS_ids,CDS_seqs,IGS_seqs,CDS_orientations
0,"[1, 3, 5, 7, 9, 10, 11, 12, 14, 16, 17, 19, 21...","[0, 2, 4, 6, 8, 13, 15, 18, 20, 22, 24, 26, 29...",[2961237444|Ga0149631_01|CDS|2961237445|-|1061...,[2961237444|Ga0149631_01|IG|IG_000001|+|1043:1...,[MNQLDALKQLTTVVADSGDIESIRQFEPQDATTNPSLILKAAALP...,"[AACGAAACTCCAGTACGG, TGATAAGCTCCTTGACCAAGCTCAA...","[False, False, True, False, True, True, True, ..."
1,"[0, 2, 3, 4, 5, 7, 9, 11, 13, 15, 17, 19, 21, ...","[1, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, ...",[2961237444|Ga0149631_01|CDS|2961237530|+|8547...,[2961237444|Ga0149631_01|IG|IG_000064|+|87422:...,[ATWYLTRAYVDQAAGRSWAALENVTRARALAPSDAGIQAYYVTLL...,"[AGGAATAATC, TACAGTCCATTGTCGTGATCATTGCGGCGATAG...","[True, True, True, True, False, True, False, T..."
2,"[0, 2, 4, 5, 6, 7, 9, 10, 12, 14, 16, 17, 18, ...","[1, 3, 8, 11, 13, 15, 19, 21, 23, 25, 28, 30, ...",[2961237444|Ga0149631_01|CDS|2961237823|-|3973...,[2961237444|Ga0149631_01|IG|IG_000289|+|400444...,[MPAAPLASLTEILARRDWQNPACTHYRRLDAHPPFASWRTVEDAR...,[CGGCGATCGTCCAGTCGGCTAGGTGATGCTCACCAAGATGCTGAC...,"[False, False, True, True, True, True, True, T..."
3,"[1, 3, 5, 7, 9, 11, 13, 14, 15, 17, 19, 21, 23...","[0, 2, 4, 6, 8, 10, 12, 16, 18, 20, 22, 24, 26...",[2961237444|Ga0149631_01|CDS|2961238380|+|9477...,[2961237444|Ga0149631_01|IG|IG_000747|+|947343...,[MDFQAIKNTARQARRYVVTMNHKAGKGHTGADLSEIDIICTLYMA...,[TCGAAAGACTCCTGCACCTGTCAGTTCCCGCCGGGCGGTTTGCCA...,"[True, True, True, True, True, True, True, Tru..."
4,"[0, 1, 3, 5, 7, 9, 10, 12, 14, 16, 18, 20, 22,...","[2, 4, 6, 8, 11, 13, 15, 17, 19, 21, 23, 25, 2...",[2961237444|Ga0149631_01|CDS|2961238402|+|9785...,[2961237444|Ga0149631_01|IG|IG_000768|+|979638...,[SESEKQQVSALSQLAAGLAGGLATGDTAGAVTGGQAGKNAVENNA...,"[GGTAGCGGGGCTCACGGCATCCCA, T, AGTAACAAAATCCCGG...","[True, True, True, True, True, True, True, Tru..."
...,...,...,...,...,...,...,...
56417,"[1, 2, 4, 6, 8, 10, 12]","[0, 3, 5, 7, 9, 11, 13]",[2828558644|Ga0392896_0312|CDS|2828559967|+|51...,[2828558644|Ga0392896_0312|IG|IG_000001|+|489:...,[MVNAQGESANTQVREMAANVNGASVENARRIEHTQLILPVDTRTP...,"[GTGGAAGCCAAACGGGCCAGG, ATATTAAGGTGTTGGAGTAGTA...","[True, True, False, False, True, False, False]"
56418,"[1, 2, 4, 6, 8]","[0, 3, 5, 7, 9]",[2828558644|Ga0392896_0320|CDS|2828559998|+|63...,[2828558644|Ga0392896_0320|IG|IG_000001|+|476:...,[MRSRNDDHQTVFSAICSHFEIERSGNVLGSVLLLFWFVGCNGSAA...,[GATCTCAAGACAATCTTGTGCGTATTTCTCATGTGTTGCGGCGCC...,"[True, False, False, False, True]"
56419,"[1, 3, 5, 7]","[0, 2, 4, 6, 8]",[2828558644|Ga0392896_0324|CDS|2828560013|+|15...,[2828558644|Ga0392896_0324|IG|IG_000001|+|1519...,[MVHLGCLDKVLKFHRKFCVPARLSTPWIFARRPKSKYRHKLANEE...,"[GGAGTGTCCTCGACCG, CTTCGGCAACTAGGAGGAAATAATAGA...","[True, True, False, False]"
56420,"[1, 3, 5, 7, 9, 10, 12, 14, 16, 18]","[0, 2, 4, 6, 8, 11, 13, 15, 17, 19]",[2828558644|Ga0392896_0325|CDS|2828560019|-|60...,[2828558644|Ga0392896_0325|IG|IG_000001|+|427:...,[MKPIRFHPVFQHYLWGGHRLANELGKPVSSADICAESWEVVDHRD...,[ATCCAAACTCCGCGGGCTGGATCAACGAGAGGGTTTGACCAAAAG...,"[False, True, True, True, True, True, False, F..."


In [9]:
input_df = input_df.head(10)
input_df['CDS_seqs'].values[0]

array(['...',
       'MEWSRRHIWAVILLSCLVFWVILATGINAVA',
       'MYSHYPAHYTFDRDSDAWQIGFHDFPEWQSACYKREDIELEAQESLLAAIAAAMDEGLPLPAPSPVQTDDLRVHLPALVALKIELHNAMLRKNTGKAALARKLGFNGGQMERLLDIGYASKVEALEQALYLLGYEVRVTISEVCQ',
       'VPKVNSRSLRFTERHVTKYAASLLFIGLSLPAAALADERYDALIRQARSGDTAPVLAYLSQRRAREGLSLQQRADYIQVAGWAGHDEEVVALW'],
      dtype=object)